<a href="https://colab.research.google.com/github/olaviinha/MusicWithChatGPT/blob/main/abc2midi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">abc2mid<font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">Music with ChatGPT</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/MusicWithChatGPT" target="_blank"><font color="#999" size="4">Github</font></a>

Converts ABC notations to MIDI.

### Usage
- When you run the Setup cell, a box will open on the right side of the screen. If you are using ChatGPT at the same time, you may copy-paste ABC notations directly from ChatGPT to that box and use the _Save the ABC notation in the right side box as .mid_ cell to immediately download whatever you currently have in the box, as a MIDI file.

- If you have files already saved as `.abc` files, you may use the _Convert .abc files to .mid files_ cell to conver them to MIDI files.

- All file or directory paths entered in this notebook should be relative to your Google Drive root (My Drive).<br>E.g. `input` value should be `Music/ABC-files/rasputin.abc` if you have a directory called _Music_ in your drive, containing a subdirectory called _ABC-files_, containing a file _rasputin.abc_.

- If you opt not to mount Drive, Files browser will open on the left side of the screen. Use it to upload/download files, and enter paths relative to that directory.<br>E.g. `input` value should be `rasputin.abc` if you uploaded `rasputin.abc` to the box on the left.

- Don't be alarmed by abc2midi warnings: most warnings can be ignored and MIDI file will work fine.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.<br>

force_setup = False
repositories = ['https://github.com/sshlien/abcmidi']
pip_packages = ''
apt_packages = ''
mount_drive = False #@ param {type:"boolean"}
skip_setup = False #@ param {type:"boolean"}
drive_mounted = False

# Download the repo from Github
import os
from google.colab import output, files
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  if apt_packages != '':
    !apt-get update && apt-get install {apt_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

drive_root = '/content/faux_drive/'
if not os.path.isdir(drive_root):
  os.mkdir(drive_root)

if len(repositories) > 0 and skip_setup == False:
  for repo in repositories:
    %cd /content/
    install_dir = fix_path('/content/'+path_leaf(repo).replace('.git', ''))
    repo = repo if '.git' in repo else repo+'.git'
    !git clone {repo}
    if os.path.isfile(install_dir+'setup.py') or os.path.isfile(install_dir+'setup.cfg'):
      !pip install -e ./{install_dir}
    if os.path.isfile(install_dir+'requirements.txt'):
      !pip install -r {install_dir}/requirements.txt

if len(repositories) == 1:
  %cd {install_dir}

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])

import time, sys, re
from datetime import timedelta
import math

# Build abc2midi
%cd /content/abcmidi
!make
!make install
%cd /content

# Create abc-notation.abc
!echo $'! Paste the ABC notation here\n! and run cell: Save the ABC notation in the right side box as .mid' > /content/abc-notation.abc

output.clear()

# Open right box
files.view('/content/abc-notation.abc')
files.view('/content/abc-notation.abc')

# !nvidia-smi
op(c.ok, 'Setup finished.', time=True)

In [ ]:
#@title # Download ABC notation in the right side box as `.mid`
#@markdown This cell will convert the current contents of the right side box to MIDI and provide a download. You may paste new content in the box and re-run this cell as many times as you like.

# #@markdown If you enter a directory path in output_dir, MIDI files will be automatically saved there. Otherwise you'll get a file download when running this cell.
# output_dir = "temp/abc/midi" #@param {type:"string"}

uniq_id = gen_id()

with open('abc-notation.abc', 'r') as f:
  abc_text = f.read()

title_match = re.search('^T:(.*)$', abc_text, re.MULTILINE)
if title_match:
  song_title = title_match.group(1).strip()
  file_out = dir_tmp+slug(song_title)+'_'+uniq_id+'.mid'
else:
  song_title = uniq_id
  file_out = dir_tmp+uniq_id+'mid'
  op(c.warn, 'No song title found, using:', uniq_id, time=True)


!abc2midi "/content/abc-notation.abc" -o "{file_out}"

print()
if os.path.isfile(file_out):
  op(c.ok, 'MIDI saved as', path_leaf(file_out), time=True)
  op(c.title, 'Downloading...', time=True)
  files.download(file_out)
else:
  op(c.fail, 'Error saving MIDI.', time=True)

In [ ]:
#@title # Convert `.abc` files to`.mid` files

#@markdown If you have saved ABC notations as `.abc` files using a text editor, use this cell to conver them to MIDI files. If a directory path is entered as `input`, all `.abc` files in the directory will be converted.

mount_drive = True #@param {type:"boolean"}
input = "" #@param {type:"string"}
output_dir = "" #@param {type:"string"}
end_session_when_done = False #@ param {type: "boolean"}


# Mount Drive
if mount_drive == True and drive_mounted == False:
  from google.colab import drive
  drive.mount('/content/drive')
  drive_root = '/content/drive/My Drive'
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root = '/content/mydrive/'
  drive_mounted = True
else:
  if mount_drive == False:
  files.view('/content/faux_drive/')
  files.view('/content/faux_drive/')
  op(c.title, 'Since you did not mount drive, please upload your files to the directory that opened on the left side of the screen. You can ignore input and output_dir and just run this cell after you have uploaded your files.')


uniq_id = gen_id()

if drive_mounted == True:
  if os.path.isfile(drive_root+input):
    inputs = [drive_root+input]
    dir_in = path_dir(drive_root+input)
  elif input != '' and os.path.isdir(drive_root+input):
    dir_in = drive_root+fix_path(input)
    # What to do if input is directory path
    inputs = glob(dir_in+'*.abc') + glob(dir_in+'*.ABC')
  elif os.path.isdir(drive_root+input) and '*' in input:
    dir_in = path_dir(drive_root+input)
    inputs = glob(drive_root+input)
  else:
    op(c.fail, 'FAIL!', 'Input should be a path to a file or a directory.')
    sys.exit('Input not understood.')
else:
  if os.path.isdir(drive_root+input):
    dir_in = drive_root+fix_path(input)
    inputs = glob(dir_in+'*.abc') + glob(dir_in+'*.ABC')
  elif os.path.isfile(drive_root+input):
    dir_in = path_dir(drive_root+input)
    inputs = [input]

# Output
if output_dir == '':
  if mount_drive is True:
    dir_out = dir_in
  else:
    dir_out = drive_root
else:
  if not os.path.isdir(drive_root+output_dir):
    os.mkdir(drive_root+output_dir)
  dir_out = drive_root+fix_path(output_dir)
  
timer_start = time.time()
total = len(inputs)

# -- DO THINGS --
for i, input in enumerate(inputs, 1):
  ndx_info = str(i)+'/'+str(total)+' '
  op(c.title, ndx_info+'Processing', input, time=True)
  file_out = dir_out+basename(input)+'__'+uniq_id+str(i).zfill(2)+'.mid'
  !abc2midi "{input}" -o "{file_out}"
  if os.path.isfile(file_out):
    op(c.ok, 'File saved as', file_out.replace(drive_root, ''), time=True)
  else:
    op(c.fail, 'ERROR saving file', file_out.replace(drive_root, ''), time=True)
# -- END THINGS --

timer_end = time.time()

print()
op(c.okb, 'Elapsed', timedelta(seconds=timer_end-timer_start), time=True)
op(c.ok, 'FIN.')

if end_session_when_done is True: end_session()